#**Stock News API**

## Data Preparation

In [4]:
import pandas as pd

In [6]:
df = pd.read_csv("nasdaq_screener_1660407220033.csv")
df

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$133.55,1.60,1.213%,3.989246e+10,United States,1999.0,865778,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,$52.76,-0.51,-0.957%,9.492679e+09,NaN,2016.0,4261157,Industrials,Metal Fabrications
2,AAL,American Airlines Group Inc. Common Stock,$15.25,0.26,1.734%,9.910150e+09,United States,NaN,21114761,Consumer Discretionary,Air Freight/Delivery Services
3,AAON,AAON Inc. Common Stock,$61.72,0.67,1.097%,3.275553e+09,United States,NaN,200322,Industrials,Industrial Machinery/Components
4,AAP,Advance Auto Parts Inc.,$202.53,5.11,2.588%,1.228138e+10,United States,NaN,530350,Consumer Discretionary,Auto & Home Supply Stores
...,...,...,...,...,...,...,...,...,...,...,...
1990,ZNH,China Southern Airlines Company Limited Common...,$27.16,0.19,0.704%,9.206405e+09,China,1997.0,8733,Consumer Discretionary,Air Freight/Delivery Services
1991,ZS,Zscaler Inc. Common Stock,$182.64,3.93,2.199%,2.590980e+10,United States,2018.0,1667742,Technology,EDP Services
1992,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,$26.00,-0.07,-0.269%,2.105306e+10,China,2016.0,1054876,Consumer Discretionary,Advertising
1993,ZTS,Zoetis Inc. Class A Common Stock,$175.67,2.49,1.438%,8.267534e+10,United States,2013.0,1185188,Health Care,Biotechnology: Pharmaceutical Preparations


In [7]:
def get_first_n_stock_tickers(n):
  return df["Symbol"].head(n).to_numpy()

In [8]:
tickers = get_first_n_stock_tickers(3)
tickers

array(['A', 'AA', 'AAL'], dtype=object)

## Marketaux

---
[Documentation](https://www.marketaux.com/documentation)

Limitations:
- 100 requests per day
- 3 articles per request



In [7]:
import requests
from bs4 import BeautifulSoup
from mlconfig import MARKETAUX_API_KEY

Get news data for the first 100 stock tickers from the stock ticker dataset (from symbol A to AMED) from Jan 2018 to Jan 2021:
- Title
- Date/Time published
- Source (News agency)
- Url
- Summary (if any)
- sentiment label (if any)

In [14]:
def get_news_data(tickers, start_date, end_date):
  news = {}
  for ticker in tickers:
    query = {
      "api_token": MARKETAUX_API_KEY,
      "symbols": ticker,
      "published_before": end_date,
      "published_after": start_date
      }
    response = requests.get("https://api.marketaux.com/v1/news/all", params = query)
    result = response.json()
    news[ticker] = result
  return news

def process_news_data(news_dict):
  df = pd.DataFrame()
  for ticker in news_dict:
    news = news_dict[ticker]
    title = list(map(lambda x: x['title'], news['data']))
    pub_date = list(map(lambda x: x['published_at'], news['data']))
    source = list(map(lambda x: x['source'], news['data']))
    url = list(map(lambda x: x['url'], news['data']))
    entities = list(map(lambda x: list(filter(lambda y: y['symbol'] == ticker, x['entities']))[0], news['data']))
    match_score = list(map(lambda x: x["match_score"], entities))
    sentiment_score = list(map(lambda x: x["sentiment_score"], entities))

    def process_text(text):
      text = text.replace('\n\n', '... ')
      text = BeautifulSoup(text, 'html.parser').get_text()
      return text
    text = list(map(lambda y: '\n'.join(list(map(lambda z: process_text(z['highlight']), y))), list(map(lambda x: x['highlights'], entities))))
    
    row = {
        "Stock": [ticker] * len(title), 
        "Title": title, 
        "Date/Time Published": pub_date, 
        "Source": source, 
        "Url": url,
        "Relevant Texts": text,
        "Sentiment Score": sentiment_score,
        "Match Score": match_score
        }
    temp = pd.DataFrame(row)
    df = df.append(temp, ignore_index = True)
  return df

In [16]:
news_dict = get_news_data(tickers, "2018-01", "2021-02")

In [17]:
process_news_data(news_dict)

,Stock,Title,Date/Time Published,Source,Url,Relevant Texts,Sentiment Score,Match Score
0,A,Agilent Technologies: Latest Guidance Increase...,2021-01-28T03:31:17.000000Z,seekingalpha.com,https://seekingalpha.com/article/4401483-agile...,Agilent Technologies: Latest Guidance Increase...,0.599400,37.514168
1,A,Global Laboratory Automation Market (2021 to 2...,2021-01-27T10:23:20.000000Z,benzinga.com,https://www.benzinga.com/pressreleases/21/01/g...,COMPETITIVE OUTLOOK... ... The key enterprises...,0.209633,39.291480
2,A,Agilent Technologies : Names Allison Ballmer S...,2021-01-18T07:10:03.000000Z,marketscreener.com,https://www.marketscreener.com/quote/stock/AGI...,Agilent Technologies Inc.\nAgilent Technologie...,0.110100,57.268425
3,AA,Notable Insider Buys Seen In The Past Week: Gr...,2021-01-30T16:10:43.000000Z,markets.businessinsider.com,https://markets.businessinsider.com/news/stock...,See also: Reddit-Fueled Rally In Mall Stock Al...,0.493900,31.419980
4,AA,Notable Insider Buys Seen In The Past Week: Gr...,2021-01-30T16:10:42.000000Z,benzinga.com,https://www.benzinga.com/news/21/01/19408291/n...,See also: Reddit-Fueled Rally In Mall Stock Al...,0.493900,37.631092
5,AA,"From Elon Musk to Reed Hastings, What CEOs Are...",2021-01-29T15:10:00.000000Z,online.wsj.com,https://www.wsj.com/articles/ceos-discuss-covi...,(Jan. 26)... Alcoa Corp. CEO Roy Harvey :... “...,0.847600,11.428745
6,AAL,The Reddit Army Bailout: Desperate Companies G...,2021-01-31T12:30:00.000000Z,bloomberg.com,https://www.bloomberg.com/news/articles/2021-0...,Yet thanks to the army of day traders who use ...,0.096917,33.930008
7,AAL,American Airlines Earnings: Still Treading Water,2021-01-30T15:51:00.000000Z,fool.com,https://www.fool.com/investing/2021/01/30/amer...,"In recent years, American Airlines Group (NASD...",-0.255829,28.873360
8,AAL,McDonald Worley Files Suit Against Robinhood F...,2021-01-30T00:00:00.000000Z,prweb.com,https://www.prweb.com/releases/mcdonald_worley...,Additional securities facing restrictions on t...,0.612400,30.177385


## Contify

---
[Documentation](https://developer.contify.com/api-documentation?python#insights-api)

In [ ]:
headers = {
    'APPID': '',
    'APPSECRET': ''
}

In [ ]:
params = {
    'company_id': ''
}

In [ ]:
contify_response = requests.get(
    'https://api.contify.com/v3/insights', headers=headers, params=params
)
companies = contify_response.json()

In [ ]:
companies